In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


import nltk
import string
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import classification_report, confusion_matrix



/home/lion/.local/lib/python3.9/site-packages/pandas/compat/_optional.py:149: UserWarning: Pandas requires version '1.3.1' or newer of 'bottleneck' (version '1.2.1' currently installed).
  warnings.warn(msg, UserWarning)


In [2]:
train = pd.read_csv('train_set.csv')
test = pd.read_csv('test_set.csv')

In [3]:
train.head()

,lang_id,text
0,xho,umgaqo-siseko wenza amalungiselelo kumaziko ax...
1,xho,i-dha iya kuba nobulumko bokubeka umsebenzi na...
2,eng,the province of kwazulu-natal department of tr...
3,nso,o netefatša gore o ba file dilo ka moka tše le...
4,ven,khomishini ya ndinganyiso ya mbeu yo ewa maana...


In [4]:
test.head()

,index,text
0,1,"Mmasepala, fa maemo a a kgethegileng a letlele..."
1,2,Uzakwaziswa ngokufaneleko nakungafuneka eminye...
2,3,Tshivhumbeo tshi fana na ngano dza vhathu.
3,4,Kube inja nelikati betingevakala kutsi titsini...
4,5,Winste op buitelandse valuta.


In [5]:
train.shape


(33000, 2)

In [6]:
test.shape

(5682, 2)

In [7]:
pd.DataFrame(train['lang_id'].value_counts())

,lang_id
xho,3000
eng,3000
nso,3000
ven,3000
tsn,3000
nbl,3000
zul,3000
ssw,3000
tso,3000
sot,3000


In [8]:
train.isnull().sum()

lang_id    0
text       0
dtype: int64

In [9]:
test.isnull().sum()

index    0
text     0
dtype: int64

In [10]:
def remove_links(text):
    '''Takes a string and removes web links from it'''
    text = re.sub(r'http\S+', '', text) # remove http links
    text = re.sub(r'bit.ly/\S+', '', text) # rempve bitly links
    return text 

punct = string.punctuation
def preprocess(sent):
    sent = remove_links(sent)
    sent = sent.lower() # lower case
    sent = re.sub('['+punct + ']+', ' ', sent) # strip punctuation
    sent = re.sub('\s+', ' ', sent.strip()) #remove any weird spacing
    sent = re.sub('([0-9]+)', '', sent) # remove numbers
    sent_token_list = [word for word in sent.split(' ')]
    sent = ' '.join(sent_token_list)
    return sent

In [11]:
df = train.copy()
df_test = test.copy()

In [12]:
df['clean_text']=df['text'].apply(lambda x: preprocess(x))
df_test['clean_text']=df_test['text'].apply(lambda x: preprocess(x))

#### Feature Engineering

In [14]:
X = df['clean_text']
y = df['lang_id']

In [15]:
le = LabelEncoder()
y_le = le.fit_transform(y)

In [16]:
#Bag of words

In [17]:
cv = TfidfVectorizer(min_df=2,max_df=0.9,ngram_range=(1, 2))
X_data = cv.fit_transform(X)
test_1 = cv.fit_transform(df_test['clean_text'])


In [46]:
test_1.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [35]:
from sklearn.feature_extraction.text import CountVectorizer
cv1 = CountVectorizer()
X_data1 = cv.fit_transform(X)
test_2 = cv.fit_transform(df_test['clean_text'])


In [47]:
X_data1.toarray()

MemoryError: Unable to allocate 51.0 GiB for an array with shape (33000, 207493) and data type float64

In [38]:
X_train,x_test,y_train,y_test = train_test_split(pd.DataFrame(X_data),y,test_size = 0.3,random_state = 18)

### Model Building

#### Logistic Regression

In [41]:
lr = LogisticRegression(random_state=18,multi_class='ovr',n_jobs=1,C=1e5,max_iter=4000)

In [42]:
lr.fit(X_train, y_train)

lr_pred = lr.predict(x_test)

print("Training Accuracy :", lr.score(X_train, y_train))
print("Validation Accuracy :", lr.score(x_test, y_test))

ValueError: setting an array element with a sequence.

In [23]:
from sklearn.metrics import classification_report
print(classification_report(y_test, lr_pred))

              precision    recall  f1-score   support

         afr       1.00      1.00      1.00       911
         eng       1.00      1.00      1.00       909
         nbl       0.99      1.00      0.99       897
         nso       1.00      1.00      1.00       894
         sot       1.00      1.00      1.00       928
         ssw       1.00      1.00      1.00       886
         tsn       1.00      1.00      1.00       871
         tso       1.00      1.00      1.00       929
         ven       1.00      1.00      1.00       843
         xho       0.99      1.00      0.99       904
         zul       0.99      0.98      0.99       928

    accuracy                           1.00      9900
   macro avg       1.00      1.00      1.00      9900
weighted avg       1.00      1.00      1.00      9900



In [24]:
ss = pd.read_csv('sample_submission.csv')
ss.head()

,index,lang_id
0,1,tsn
1,2,nbl


In [25]:
# lr1_pred = lr.predict(test_1)

#### Random Forest Classifier

In [26]:
rfc = RandomForestClassifier()

In [27]:
rfc.fit(X_train, y_train)

rfc_pred = rfc.predict(x_test)

print("Training Accuracy :", rfc.score(X_train, y_train))
print("Validation Accuracy :", rfc.score(x_test, y_test))

Training Accuracy : 1.0
Validation Accuracy : 0.9870707070707071


In [28]:
from sklearn.metrics import classification_report
print(classification_report(y_test, rfc_pred))

              precision    recall  f1-score   support

         afr       1.00      1.00      1.00       911
         eng       0.99      1.00      1.00       909
         nbl       0.98      0.95      0.97       897
         nso       1.00      1.00      1.00       894
         sot       1.00      1.00      1.00       928
         ssw       0.98      0.98      0.98       886
         tsn       1.00      1.00      1.00       871
         tso       1.00      1.00      1.00       929
         ven       1.00      1.00      1.00       843
         xho       0.96      0.99      0.97       904
         zul       0.95      0.95      0.95       928

    accuracy                           0.99      9900
   macro avg       0.99      0.99      0.99      9900
weighted avg       0.99      0.99      0.99      9900



In [29]:
submission_df = pd.DataFrame(test['index'])
submission_df['lang_id'] = rfc.predict(test_1)
submission_df.to_csv('rfc.csv', index=False)

ValueError: X has 26977 features, but RandomForestClassifier is expecting 207493 features as input.